In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import re
import os
from itertools import chain
import glob
import seaborn as sns
import matplotlib.font_manager as font_manager
import seaborn as sns
import matplotlib as mpl
from pytorch_tabnet.tab_model import TabNetRegressor
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.linear_model import SGDRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor, Pool
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import RandomizedSearchCV
from collections import Counter
from sklearn.model_selection import GridSearchCV
from pandas import read_csv
from datetime import datetime
from sklearn.metrics import mean_absolute_percentage_error
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pandas.plotting import register_matplotlib_converters
import warnings
warnings.filterwarnings('ignore')
register_matplotlib_converters()
sns.set_theme()
mpl.matplotlib_fname()
font_dirs = ['D:\\Software\\Anaconda\\Lib\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf' ]
font_files = font_manager.findSystemFonts(fontpaths=font_dirs)
for font_file in font_files:
    font_manager.fontManager.addfont(font_file)
mpl.font_manager._rebuild()
result = font_manager.findfont("Rounded Mplus 1c")
print(result)
mpl.rc('font', family='Rounded Mplus 1c')


le = LabelEncoder()
ss = StandardScaler()

D:\Software\Anaconda\Lib\site-packages\matplotlib\mpl-data\fonts\ttf\MPLUSRounded1c-Regular.ttf


In [2]:
names = ["Gradient_Boosting", "Decision_Tree", "Extra_Trees", "Random_Forest","XGBoost","LGBMRegressor"]
models = [
    GradientBoostingRegressor(),
    DecisionTreeRegressor(max_depth=5),
    ExtraTreesRegressor(),
    RandomForestRegressor(),
    XGBRegressor(),
    LGBMRegressor()]

In [4]:
df = pd.read_csv('Kaiho_merged_data.csv')

In [ ]:
df_group_mean =  df.groupby(['Engine model_エンジン型式']).agg(mean = ('Thành tích bán ra_販売実績','mean')).reset_index()

In [5]:
df.columns

Index(['Vị trí_ロケーション', 'Ngày sản xuất_生産日', 'Tháng sản xuất_生産月',
       'Năm sản xuất_生産年', 'Nv sản xuất_生産担当者', 'Số nhập kho_入庫番号',
       'Nguồn nhập hàng_仕入先', 'Mã loại phụ tùng_商品コード', 'Tên phụ tùng_商品名',
       'Số nhập kho(phụ tùng)_入庫番号 (部品)', 'Tên xe_車名', 'Model xe_車輌型式',
       'Engine model_エンジン型式', 'Số lượng_数量_PARTS', 'Tổng giá nhập_仕入合計',
       'Số phút công_標準工数', 'Kết quả_生産高',
       'Ước tính giá bán ra tại thời điểm nhập hàng_仕入時販売予測額',
       'Ước tính giá bán ra tại thời điểm sản xuất_生産時販売予測額',
       'Số lượng bán ra_販売数量', 'Thành tích bán ra_販売実績', 'Khách hàng_仕向先',
       'Truyền động 1_駆動1', 'TM_TM', 'Truyền động 2_駆動2', 'Chi tiết_詳細',
       'Số km đã chạy_走行距離', 'Năm sản xuất_年式', 'Tình trạng engine_エンジン状態',
       'Rank_ランク', 'Loại xe_車種', 'Tình trạng sản xuất_未生産',
       'Ước tính giá bán ra tại thời điểm nhập hàng_仕入時販売予測額.1',
       'Số chỉ định loại_型式指定番号', 'Số phân loại danh mục_類別区分番号',
       'Đơn giá mua_仕入単価', 'Nhà sản xuất_メーカー', 'Giờ làm việc

In [6]:
df['date_time']  = df['Năm bán hàng_売上年'].astype(str) +'-'+df['Tháng bán hàng_売上月'].astype(str) 

In [65]:
df_copy = df.copy()

In [66]:
df_copy = pd.merge(df_copy,df_group_mean, on='Engine model_エンジン型式',how='outer')
df_copy

,Vị trí_ロケーション,Ngày sản xuất_生産日,Tháng sản xuất_生産月,Năm sản xuất_生産年,Nv sản xuất_生産担当者,Số nhập kho_入庫番号,Nguồn nhập hàng_仕入先,Mã loại phụ tùng_商品コード,Tên phụ tùng_商品名,Số nhập kho(phụ tùng)_入庫番号 (部品),...,Số body_車体番号,CONTAINER_CONTAINER,Tên khách hàng_売上先名,Phân loại mang đến/đón_持込・引取区分,Tiền công_工賃,D/G_Ｄ／Ｇ,Phân loại bán hàng_売上分類,Đang tải nguồn_積込元,date_time,mean
0,会宝産業本社,17,11,2015,山下大輝,'0000000302676,西田 敏勝,'110010,ノーズカット（個別）,'0000007988203,...,'1001019,NaN,"一番オート OOO ""LIGHT VL""",NaN,0.0,G,NaN,会宝産業本社,2016-3,36000.000000
1,会宝産業本社,18,10,2018,山下宣浩,'0000000374316,日本梱包運輸倉庫株式会社,'100000,エンジン,'0000004270345,...,'1005616,40HC,木戸クンク・ジャイブ（ＥＫ３）,NaN,0.0,G,NaN,会宝産業本社,2018-10,36000.000000
2,会宝産業本社,18,10,2018,西川航平,'0000000374316,日本梱包運輸倉庫株式会社,'110010,ノーズカット（個別）,'0000004270519,...,'1005616,40HC,木戸クンク・ジャイブ（ＥＫ３）,NaN,0.0,G,NaN,会宝産業本社,2018-10,36000.000000
3,会宝産業本社,6,6,2019,中嶋 裕,'0000000400132,京谷 宏,'005020,ＣＫＤ／コンプリート,'0000004451332,...,'1006268,40HC,FADEEV EVGENY,NaN,0.0,G,NaN,会宝産業本社,2019-7,36000.000000
4,会宝産業本社,3,2,2015,中嶋 修,'0000000303771,ｴｻﾞｼ自動車,'100000,エンジン,'0000007739581,...,'6062039,40HC,木戸クンク・ジャイブ（ＥＫ３）,NaN,0.0,G,NaN,会宝産業本社,2015-2,38349.244713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20423,会宝産業本社,8,11,2012,坂尻僚平,'0000000298731,北陸スバル自動車カースポット,'100000,エンジン,'0000007113103,...,'109446,20,FADEEV EVGENY,NaN,0.0,G,NaN,会宝産業本社,2012-11,50000.000000
20424,会宝産業本社,27,9,2012,西野昌明,'0000000299537,数馬石油(株),'100000,エンジン,'0000007078345,...,'131992,20,"VINAKO CO., LTD.",NaN,0.0,G,NaN,会宝産業本社,2012-11,45000.000000
20425,会宝産業本社,31,10,2012,中嶋 修,'0000000300812,オートまつもと,'100000,エンジン,'0000007105617,...,'0268644,20,FADEEV EVGENY,NaN,0.0,G,NaN,会宝産業本社,2012-10,25000.000000
20426,会宝産業本社,7,12,2012,瀬戸誠路,'0000000301982,アサヒ自動車整備工場,'100000,エンジン,'0000007135026,...,'097134,20,"VINAKO CO., LTD.",NaN,0.0,G,NaN,会宝産業本社,2012-12,25000.000000


In [123]:
list_feature = [
                'Tên phụ tùng_商品名',
#                 'Tên xe_車名',
                'Model xe_車輌型式',
                'Engine model_エンジン型式',
                'Thành tích bán ra_販売実績',
                'Tháng bán hàng_売上月',
                 'mean',
#                 'Năm bán hàng_売上年'
#                 'date_time',
#                'D/G_Ｄ／Ｇ',
#                'Loại xe_車種'
               ]

In [124]:
# df_group_month = df_copy.groupby(list_feature).agg(revenue = ('Thành tích bán ra_販売実績','sum')).reset_index()
df_group_month = df_copy[list_feature]

In [125]:
# df_group_month = df_group_month.sort_values(by=['date_time'])
# df_group_month

In [126]:
for col in df_group_month.columns[df_group_month.dtypes == object]:
    print(col, df_group_month[col].nunique())
    df_group_month[col] = df_group_month[col].fillna("unknown")
    df_group_month[col] = le.fit_transform(df_group_month[col].values)

for col in df_group_month.columns[df_group_month.dtypes == 'float64']:
    df_group_month.fillna(df_group_month[col].mean(), inplace=True)

Tên phụ tùng_商品名 12
Model xe_車輌型式 1587
Engine model_エンジン型式 536


In [127]:
# df_group_month.date_time.unique()
df_group_month

,Tên phụ tùng_商品名,Model xe_車輌型式,Engine model_エンジン型式,Thành tích bán ra_販売実績,Tháng bán hàng_売上月,mean
0,1,370,386,12000.0,3,36000.000000
1,0,400,386,30000.0,10,36000.000000
2,1,400,386,12000.0,10,36000.000000
3,8,400,386,90000.0,7,36000.000000
4,0,73,156,30000.0,2,38349.244713
...,...,...,...,...,...,...
20423,0,363,522,50000.0,11,50000.000000
20424,0,1323,305,45000.0,11,45000.000000
20425,0,46,58,25000.0,10,25000.000000
20426,0,858,159,25000.0,12,25000.000000


# Without Scaler

In [128]:
target = df_group_month['Thành tích bán ra_販売実績']
# target = df_group_mean['Thành tích bán ra_販売実績']

In [129]:
data = df_group_month.drop(columns=['Thành tích bán ra_販売実績'])

In [130]:
X_train, X_test, y_train, y_test =  train_test_split(data, target, test_size= 0.3, random_state=42)

In [131]:
X_train

,Tên phụ tùng_商品名,Model xe_車輌型式,Engine model_エンジン型式,Tháng bán hàng_売上月,mean
7417,1,23,1,3,42361.459016
5118,0,1262,421,5,39805.870206
2231,0,53,31,5,94884.669789
6309,1,290,535,12,23507.500000
18453,11,1026,413,2,78529.411765
...,...,...,...,...,...
11284,1,33,138,10,52724.157895
11964,0,45,228,8,17828.415301
5390,0,1068,421,9,39805.870206
860,0,1047,178,10,38270.736364


In [132]:
rmse = []
mape = []
r_squared = []
for  model in models:
    model.fit(X_train, y_train)
    y_pred  = model.predict(X_test)
#     print(y_pred)
    rmse_score = mean_squared_error(y_test, y_pred,squared=False)
    mape_score = np.mean(np.abs((y_test - y_pred)/y_test)*100)
    print(mape_score)
#     mape_score = mean_absolute_percentage_error(y_test,y_pred)
    r_squared_score  = model.score(X_test,y_test)
    r_squared.append(r_squared_score)
    rmse.append(rmse_score)
    mape.append(mape_score)

21.27647320061412
19.495617852145617
17.67357315631125
17.089276424985552
19.908330612303576
19.039674377404268


# With Scaler

In [23]:
df_scale = pd.DataFrame(ss.fit_transform(df_group_month),columns=df_group_month.columns)

In [24]:
target_scaler = df_scale['Thành tích bán ra_販売実績']

In [25]:
target_scaler

0       -0.697563
1       -0.200797
2       -0.752760
3       -0.697563
4       -0.752760
           ...   
20423   -0.697563
20424   -0.932148
20425   -0.932148
20426   -0.890750
20427   -0.890750
Name: Thành tích bán ra_販売実績, Length: 20428, dtype: float64

In [26]:
data = df_scale.drop(columns=['Thành tích bán ra_販売実績'])

In [27]:
X_train, X_test, y_train, y_test =  train_test_split(data, target, test_size= 0.3, random_state=42)

In [28]:
y_test

11876     35000.0
6735      20000.0
12902     30000.0
19893     42000.0
20269     10000.0
           ...   
17288     30000.0
16615     25000.0
7638     100000.0
8361     180000.0
15451      8000.0
Name: Thành tích bán ra_販売実績, Length: 6129, dtype: float64

In [29]:
y_pred

array([ 37889.1083147 ,  23307.18289944,  37737.90405802, ...,
       139063.31555807, 126280.83128107,   8815.57666608])

In [30]:
rmse = []
mape = []
r_squared = []
for  model in models:
    model.fit(X_train, y_train)
    y_pred  = model.predict(X_test)
#     print(y_pred)
    rmse_score = mean_squared_error(y_test, y_pred,squared=False)
    mape_score = np.mean(np.abs((y_test - y_pred)/y_test)*100)
    print(mape_score)
#     mape_score = mean_absolute_percentage_error(y_test,y_pred)
    r_squared_score  = model.score(X_test,y_test)
    r_squared.append(r_squared_score)
    rmse.append(rmse_score)
    mape.append(mape_score)

30.019123251756252
29.51217812572254
16.885073351359257
16.747535266819057
20.73886563880307
21.627152411092542


In [31]:
df_score = pd.DataFrame()
df_score['MODELS'] = names
df_score['R_SQUARED'] = r_squared
df_score['RMSE'] = rmse
df_score['MAPE'] = mape
df_score['RMSE'] = round(df_score['RMSE'],3)
df_score['MAPE'] = round(df_score['MAPE'],3)
df_score['R_SQUARED'] = round(df_score['R_SQUARED'],3)

df_score

,MODELS,R_SQUARED,RMSE,MAPE
0,Gradient_Boosting,0.748,17335.679,30.019
1,Decision_Tree,0.680,19550.455,29.512
2,Extra_Trees,0.818,14721.415,16.885
3,Random_Forest,0.824,14497.032,16.748
4,XGBoost,0.835,14038.772,20.739
5,LGBMRegressor,0.818,14750.074,21.627


In [220]:
y_pred

array([87565.98177763, 90488.39485411, 83054.88794164, ...,
       28129.22654704, 10445.3873728 , 44312.15762412])

In [221]:
target_scaled = ss.fit_transform(df_group_month[['revenue']])
l = ss.inverse_transform(y_pred)
l

array([3.77504654e+09, 3.90103291e+09, 3.58057152e+09, ...,
       1.21270480e+09, 4.50347589e+08, 1.91035729e+09])

In [222]:
k = ss.inverse_transform(y_test)
k

array([3.87997802e+09, 3.87997802e+09, 3.23332216e+09, ...,
       1.07780263e+09, 4.31146765e+08, 1.72445849e+09])

In [223]:
mean_squared_error(l,k,squared= False)

786748157.4494454

In [224]:
model  = XGBRegressor()

In [225]:
model.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=12,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [226]:
X_test

,Tên phụ tùng_商品名,Tên xe_車名,Model xe_車輌型式,Engine model_エンジン型式,Đơn giá bán hàng_売上単価,date_time
15277,-0.410486,-1.056367,-1.102525,-1.105632,1.370901,1.223949
6444,2.703361,-1.160204,0.388760,-1.175921,1.370901,-0.506360
10085,-0.410486,-1.104292,1.318080,-0.837257,0.974679,0.223614
11011,-0.410486,-0.121832,-0.109793,-1.073682,-0.213985,0.412867
5155,-0.410486,-1.855115,0.119804,0.562129,-0.213985,-0.695612
...,...,...,...,...,...,...
6186,-0.410486,-1.024417,-1.323375,0.376823,-0.213985,-0.533396
12400,-0.410486,0.621004,-0.934154,2.095703,-0.213985,0.656191
14071,-0.410486,0.405342,0.570251,1.533393,-0.346059,0.980624
8395,-0.021255,-0.145795,1.759780,0.798555,-0.742280,-0.181927


In [227]:
y_pred = model.predict(X_test)

In [228]:
mean_absolute_percentage_error(y_test,y_pred)

0.2102952441410314

In [259]:
test = ['エンジン','ﾗﾌｪｽﾀ','NB30','MR20DE','40000.0','2019-9']
encode = le.fit_transform(np.array([[1, 2], [3, 1], [3, 2]]))
test_encode = encode.transform(test)

ValueError: y should be a 1d array, got an array of shape (3, 2) instead.

In [257]:
test_encode

array([4, 5, 3, 2, 1, 0])

In [251]:
model.predict(np.array(test_encode).reshape(1,len(test_encode)))

TypeError: object of type 'LabelEncoder' has no len()

In [245]:
from sklearn.preprocessing import StandardScaler
data = np.array([[1, 2], [3, 1], [3, 2]])
scaler = StandardScaler()
print(scaler.fit_transform(data))

[[-1.41421356  0.70710678]
 [ 0.70710678 -1.41421356]
 [ 0.70710678  0.70710678]]


In [246]:
data.shape

(3, 2)